In [1]:
import warnings

import pytest
import numpy as np

import TidalPy
TidalPy.test_mode = True


from TidalPy.radial_solver import radial_solver as rs_old
from TidalPy.RadialSolver import radial_solver

frequency = 1.0 / (86400.0 * 0.5)
N = 100
radius_array = np.linspace(0.1, 6000.e3, N)
density_array = 3500. * np.ones_like(radius_array)
bulk_modulus_array = 1.0e11 * np.ones_like(radius_array)
viscosity_array = 1.0e20 * np.ones_like(radius_array)
shear_array = 5.0e10 * np.ones_like(radius_array)

from TidalPy.rheology.models import Maxwell
complex_shear_modulus_array = np.empty(N, dtype=np.complex128)
max_rho = Maxwell()
max_rho.vectorize_modulus_viscosity(frequency, shear_array, viscosity_array, complex_shear_modulus_array)


from TidalPy.utilities.spherical_helper import calculate_mass_gravity_arrays
volume_array, mass_array, gravity_array = calculate_mass_gravity_arrays(radius_array, density_array)

planet_bulk_density = np.sum(mass_array) / np.sum(volume_array)
upper_radius_by_layer = (radius_array[-1],)

is_solid = False
is_static =  False
is_incompressible= False
method=2
degree_l=2
use_kamata=True
is_solid_by_layer = (is_solid,)
is_static_by_layer = (is_static,)
is_incompressible_by_layer = (is_incompressible,)

index_by_layer = (~np.isnan(radius_array),)

integration_rtol = 1.0e-6
integration_atol = 1.0e-10

def test_RadialSolver():
    
    out = radial_solver(
        radius_array, density_array, gravity_array, bulk_modulus_array, complex_shear_modulus_array,
        frequency, planet_bulk_density, 
        is_solid_by_layer, is_static_by_layer, is_incompressible_by_layer, upper_radius_by_layer,
        degree_l=degree_l, solve_for=None, use_kamata=use_kamata,
        integration_method=method, integration_rtol=integration_rtol, integration_atol=integration_atol,
        scale_rtols_by_layer_type=False,
        max_num_steps=50_000_000, expected_size=250, max_step=0, max_ram_MB=4500,
        limit_solution_to_radius=True, verbose=True, nondimensionalize=True)
    
    assert out.success
    assert type(out.message) is str
    assert type(out.result) is np.ndarray
    assert out.result.shape == (6, N)
    assert np.all(np.isnan(out.result) == False)

def test_rs_old():
    
    out = rs_old(
        radius_array, complex_shear_modulus_array, bulk_modulus_array,
        density_array, gravity_array, frequency, planet_bulk_density,
        is_solid_by_layer=is_solid_by_layer,
        is_static_by_layer=is_static_by_layer,
        indices_by_layer=index_by_layer,
        order_l=2,
        surface_boundary_condition=None, solve_load_numbers=False,
        use_kamata=use_kamata,
        integrator='numba', integration_method='RK45',
        integration_rtol=integration_rtol, integration_atol=integration_atol,
        verbose=True, nondimensionalize=True, incompressible=is_incompressible
        )

2024-01-23 15:47:33(+00:00:02::846728) - INFO     : TidalPy initializing...
2024-01-23 15:47:33(+00:00:02::848753) - INFO     : Output directory: C:\Users\jrenaud\Documents\Software\TidalPy\Tests\TidalPy-Run_20240123-1547
2024-01-23 15:47:33(+00:00:02::848753) - INFO     : TidalPy initialization complete.


In [2]:
test_rs_old()

Solving Layer 1 (using CPUDispatcher(<function nbrk_solver at 0x000001E442336480>) with 1)...

Integration Done!
Collapsing solutions...


C:\Users\jrenaud\AppData\Local\anaconda3\envs\dev311\Lib\site-packages\numba\core\ir_utils.py:2174: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'gravity_at_interfaces' of function 'collapse_solutions'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\AppData\Local\anaconda3\envs\dev311\Lib\site-packages\TidalPy\radial_solver\numerical\collapse\generalized_collapse.py", line 23:
@njit(cacheable=True)
def collapse_solutions(
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
C:\Users\jrenaud\AppData\Local\anaconda3\envs\dev311\Lib\site-packages\numba\core\ir_utils.py:2174: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'liquid_density_at_interfaces' of function 'collapse_solution

Re-dimensionalizing Radial Functions.
Done!


In [3]:
test_RadialSolver()

Starting integration
Collapsing radial solutions for "0" solver.
Error encountered while applying surface boundary condition. ZGESV code: 2
The solutions may not be valid at the surface.


AssertionError: 